                                 NHIỆM VỤ 1: ĐỌC VÀ LÀM SẠCH DỮ LIỆU


In [1]:
## 1. Đọc dữ liệu từ thư mục data_raw
##Dữ liệu gốc được giữ nguyên, không chỉnh sửa, chỉ đọc để xử lý.

import pandas as pd
import numpy as np

df_patient = pd.read_csv("../data_raw/patient_info.csv")
df_vaccine = pd.read_csv("../data_raw/vaccination_record.csv")
df_reaction = pd.read_csv("../data_raw/reaction_log.csv")

In [2]:
## 2. Khảo sát cấu trúc dữ liệu
##Kiểm tra số dòng, số cột, kiểu dữ liệu và các lỗi định dạng ban đầu.
def khao_sat(ten_bang, df):
    print(f'\n=== {ten_bang} ===')
    print('Số dòng, số cột:', df.shape)
    print('\nKiểu dữ liệu:')
    print(df.dtypes)
    print('\n5 dòng đầu:')
    print(df.head())

khao_sat('PATIENT_INFO', df_patient)
khao_sat('VACCINATION_RECORD', df_vaccine)
khao_sat('REACTION_LOG', df_reaction)




=== PATIENT_INFO ===
Số dòng, số cột: (40, 3)

Kiểu dữ liệu:
patient_id    object
age_group     object
gender        object
dtype: object

5 dòng đầu:
  patient_id age_group   gender
0      BN500      6-17  Female 
1      bn501       0-5        m
2      bn502       50+     nam 
3     BN503       6-17     nam 
4      BN504     18_49     male

=== VACCINATION_RECORD ===
Số dòng, số cột: (120, 4)

Kiểu dữ liệu:
patient_id          object
vaccine_type        object
dose_number         object
vaccination_date    object
dtype: object

5 dòng đầu:
  patient_id vaccine_type dose_number vaccination_date
0      BN535          DPT         1st       2024-02-23
1      bn520     covid19            3       2024-03-07
2     BN528           DPT           2       23/03/2024
3      bn529        COVID           3       2024-01-05
4      BN508     covid19            1       01/05/2024

=== REACTION_LOG ===
Số dòng, số cột: (80, 4)

Kiểu dữ liệu:
patient_id       object
reaction_type    object
severity    

In [10]:
##3.Chuẩn hoá mã và nhãn quan trọng: patient_id, gender (M/F), age_group, vaccine_type.

#Chuẩn hóa patient_id:
def chuan_hoa_patient_id(bang):
    bang['patient_id'] = (
        bang['patient_id']
        .astype(str)
        .str.strip()
        .str.upper()
    )
    return bang
df_patient = chuan_hoa_patient_id(df_patient)
df_vaccine = chuan_hoa_patient_id(df_vaccine)
df_reaction = chuan_hoa_patient_id(df_reaction)


#Chuẩn hóa gendar(M/F):
map_gioi_tinh = {
    'NAM': 'M', 'MALE': 'M', 'M': 'M', '1': 'M',
    'NỮ': 'F', 'NU': 'F', 'FEMALE': 'F', 'F': 'F', '0': 'F'
}

df_patient['gender'] = (
    df_patient['gender']
    .astype(str)
    .str.strip()
    .str.upper()
    .map(map_gioi_tinh)
)


#Chuẩn hóa age_group
age_map = {
    "0_5": "0-5", "0–5": "0-5", "0-5": "0-5",
    "6_12": "6-12", "6-12": "6-12",
    "6_17": "6-17", "13_17": "13-17", "13-17": "13-17",
    "18_49": "18-49", "18-49": "18-49",
    "50+": "50+", "50plus": "50+", "50 plus": "50+",
    "65+": "65+", "65 plus": "65+"
}
df_patient["age_group"] = (
    df_patient["age_group"]
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(age_map)
)


#Chuẩn hóa loại vaccine
vaccine_map = {
    "d.p.t":"DPT",
    "dpt": "DPT",
    "covid19":"Covid-19",
    "covid-19":"Covid-19",
    "COVID": "Covid-19",
    "covid": "Covid-19",
    "mmr": "MMR"

}

df_vaccine["vaccine_type"] = (
    df_vaccine["vaccine_type"]
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(vaccine_map)
)

#Chuẩn hoá reaction_type
df_reaction['reaction_type'] = (
    df_reaction['reaction_type']
    .astype(str)
    .str.strip()
    .str.lower()
)

map_reaction = {
    'đau tay': 'Pain',
    'dau tay': 'Pain',

    'sốt': 'Fever',
    'sot': 'Fever',

    'không phản ứng': 'None',
    'khong phan ung': 'None',

    '!@#': np.nan,
    'nan': np.nan
}

df_reaction['reaction_type'] = df_reaction['reaction_type'].replace(map_reaction)

#Chuẩn hoá severity về thang 0–3
df_reaction['severity'] = (
    df_reaction['severity']
    .astype(str)
    .str.strip()
    .str.lower()
)

map_severity = {
    '0': 0, '1': 1, '2': 2, '3': 3,
    'low': 1,
    'moderate': 2,
    'high': 3,
    'severe': 3,
    'nặng': 3,
    'nang': 3
}

df_reaction['severity'] = df_reaction['severity'].replace(map_severity)
df_reaction['severity'] = (
    pd.to_numeric(df_reaction['severity'], errors='coerce')
    .astype('Int64')
)


Chú thích: theo mô tả dữ liệu, file patient_info.csv được kỳ vọng có trường full_name. Tuy nhiên, trong dữ liệu thực tế nhận được, file này không chứa cột full_name. Do đó, người làm không thực hiện bước làm sạch đối với trường này.

In [4]:
##4. Chuẩn hoá trường số & ngày tháng: dose_number → số nguyên; vaccination_date & report_date → YYYY-MM-DD.

# Chuẩn hoá dose_number → số nguyên
import pandas as pd

df_vaccine["dose_number"] = (
    df_vaccine["dose_number"]
    .astype(str)
    .str.extract(r"(\d+)")
    .astype("Int64")
)

# Chuẩn hoá vaccination_date → YYYY-MM-DD
df_vaccine["vaccination_date"] = pd.to_datetime(
    df_vaccine["vaccination_date"],
    errors="coerce",
    dayfirst=True
)
df_vaccine["vaccination_date"] = df_vaccine["vaccination_date"].dt.strftime("%Y-%m-%d")


# Chuẩn hoá report_date → YYYY-MM-DD
df_reaction["report_date"] = pd.to_datetime(
    df_reaction["report_date"],
    errors="coerce",
    dayfirst=True
)
df_reaction["report_date"] = df_reaction["report_date"].dt.strftime("%Y-%m-%d")


C:\Users\thuy linh\AppData\Local\Temp\ipykernel_23488\3361595502.py:14: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_vaccine["vaccination_date"] = pd.to_datetime(
C:\Users\thuy linh\AppData\Local\Temp\ipykernel_23488\3361595502.py:23: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_reaction["report_date"] = pd.to_datetime(


In [12]:
##5. Xử lý dữ liệu thiếu và giá trị bất thường trong tất cả các bảng.
gia_tri_rong = ['', 'None', 'none', 'NULL', 'null', 'NA', 'na', ' ']

for df in [df_patient, df_vaccine, df_reaction]:
    df.replace(gia_tri_rong, np.nan, inplace=True)

# Bảng PATIENT 
# patient_id thiếu → không thể sử dụng → loại bỏ
df_patient = df_patient.dropna(subset=['patient_id'])


# Bảng VACCINE 
# patient_id thiếu → loại bỏ
df_vaccine = df_vaccine.dropna(subset=['patient_id'])

# vaccination_date lỗi → NaT
df_vaccine['vaccination_date'] = pd.to_datetime(
    df_vaccine['vaccination_date'], errors='coerce'
)

# Bảng REACTION
# patient_id thiếu → loại bỏ
df_reaction = df_reaction.dropna(subset=['patient_id'])


# Báo cáo dữ liệu thiếu sau xử lý
print('THIẾU DỮ LIỆU SAU XỬ LÝ')

print('\nPATIENT')
print(df_patient.isna().sum())

print('\nVACCINE')
print(df_vaccine.isna().sum())

print('\nREACTION')
print(df_reaction.isna().sum())



THIẾU DỮ LIỆU SAU XỬ LÝ

PATIENT
patient_id    0
age_group     0
gender        0
dtype: int64

VACCINE
patient_id           0
vaccine_type         0
dose_number         14
vaccination_date    61
dtype: int64

REACTION
patient_id        0
reaction_type    35
severity          9
report_date      39
dtype: int64


In [15]:
# 6. Kiểm tra tính hợp lệ logic giữa các bảng dữ liệu

# 6.1 Phát hiện liều tiêm trùng lặp (cùng bệnh nhân – cùng liều – cùng ngày)
duplicate_dose = df_vaccine[
    df_vaccine.duplicated(
        subset=['patient_id', 'dose_number', 'vaccination_date'],
        keep=False
    )
]
print("Số dòng liều tiêm trùng lặp:", len(duplicate_dose))


# 6.2 Kiểm tra phản ứng được báo trước ngày tiêm (lỗi logic thời gian)
ngay_tiem_dau = (
    df_vaccine
    .groupby('patient_id')['vaccination_date']
    .min()
)

df_reaction['vaccination_date_min'] = (
    df_reaction['patient_id'].map(ngay_tiem_dau)
)

reaction_sai_ngay = df_reaction[
    (df_reaction['report_date'].notna()) &
    (df_reaction['vaccination_date_min'].notna()) &
    (df_reaction['report_date'] < df_reaction['vaccination_date_min'])
]

print("Số phản ứng báo trước ngày tiêm:", len(reaction_sai_ngay))


# 6.3 Phát hiện phản ứng nhưng không có dữ liệu tiêm tương ứng
reaction_khong_co_tiem = df_reaction[
    ~df_reaction['patient_id'].isin(df_vaccine['patient_id'])
]

print("Phản ứng nhưng không có dữ liệu tiêm:", len(reaction_khong_co_tiem))


# 6.4 Phát hiện dữ liệu tiêm nhưng không có thông tin bệnh nhân
vaccine_khong_co_patient = df_vaccine[
    ~df_vaccine['patient_id'].isin(df_patient['patient_id'])
]

print("Tiêm nhưng không có thông tin bệnh nhân:", len(vaccine_khong_co_patient))


Số dòng liều tiêm trùng lặp: 27
Số phản ứng báo trước ngày tiêm: 10
Phản ứng nhưng không có dữ liệu tiêm: 2
Tiêm nhưng không có thông tin bệnh nhân: 0


In [16]:
#7. Đồng bộ patient_id để đảm bảo khả năng merge ở các nhiệm vụ sau.
tap_patient_id = set(df_patient['patient_id']) \
    | set(df_vaccine['patient_id']) \
    | set(df_reaction['patient_id'])

print('Tổng số patient_id sau đồng bộ:', len(tap_patient_id))


Tổng số patient_id sau đồng bộ: 40


In [17]:
#8. Xuất các file sạch
df_patient.to_csv('../data_clean/patient_info_clean.csv', index=False)
df_vaccine.to_csv('../data_clean/vaccination_record_clean.csv', index=False)
df_reaction.to_csv('../data_clean/reaction_log_clean.csv', index=False)


